In [1]:
import ee
import geemap
import datetime
from shapely.geometry import shape

# Initialize Earth Engine and Geemap
ee.Initialize(project='servir-ee')

In [2]:
Map = geemap.Map(center=[6.746, -57.98], zoom=14)

In [3]:
# ----------------------------- #
#       ANALYSIS PARAMETERS      #
# ----------------------------- #
roi = ee.Geometry.BBox(-58.024429, 6.729363, -57.935249, 6.763412)
start_date = '1984-03-16'
end_date = '2024-12-31'
building_resolution = 5  # meters

# ----------------------------- #
#         DATA PROCESSING        #
# ----------------------------- #
print("🚀 Loading datasets...")
jrc = (ee.ImageCollection("JRC/GSW1_4/MonthlyHistory")
      .filterBounds(roi)
      .filterDate(start_date, end_date)
      .select('water'))

srtm = ee.Image("USGS/SRTMGL1_003").clip(roi)
permanent_water = ee.Image("JRC/GSW1_4/GlobalSurfaceWater").select("occurrence").gte(90)

# Convert buildings to raster
buildings = (ee.FeatureCollection("GOOGLE/Research/open-buildings/v3/polygons")
            .filterBounds(roi)
            .map(lambda f: f.centroid()))
building_raster = (ee.Image().byte()
                  .paint(buildings, 1)
                  .reproject(crs='EPSG:4326', scale=building_resolution)
                  .clip(roi))

# ----------------------------- #
#         FLOOD ANALYSIS        #
# ----------------------------- #
print("🔍 Calculating flood frequency...")
flood_freq = (
    jrc.map(lambda img: img.eq(2).And(permanent_water.Not())).sum()
    .divide(jrc.map(lambda img: img.gt(0).And(permanent_water.Not())).sum())
    .multiply(100)
    .rename('flood_frequency')
)

print("🌊 Identifying flood-prone areas...")
flood_prone = (
    flood_freq.updateMask(
        ee.Terrain.slope(srtm).lt(5).And(srtm.lt(10))
    )
    .focal_max(radius=5, units='meters')
)

# ----------------------------- #
#         BUILDING COUNTS       #
# ----------------------------- #
print("🏠 Counting buildings...")
flooded_buildings = building_raster.updateMask(flood_prone.gt(0))

# Get statistics using Earth Engine reducers
def get_stats(image):
    stats = image.reduceRegion(
        reducer=ee.Reducer.sum(),
        geometry=roi,
        scale=building_resolution,
        maxPixels=1e13
    )
    return stats.get('constant').getInfo()

total_buildings = get_stats(building_raster) * 1.18  # Calibration factor
flooded_count = get_stats(flooded_buildings) * 1.18

print(f"\n📊 Final Results:")
print(f"Total buildings: {total_buildings:,.0f}")
print(f"Flooded buildings: {flooded_count:,.0f}")

# ----------------------------- #
#        VISUALIZATION         #
# ----------------------------- #
print("\n🎨 Generating visualization...")

# Visualization parameters
flood_vis = {'min': 0, 'max': 100, 'palette': ['0000FF']}
building_vis = {'min': 0, 'max': 1, 'palette': ['FF0000']}
flooded_vis = {'min': 0, 'max': 1, 'palette': ['FFFF00']}

# Add layers
Map.addLayer(flood_prone, flood_vis, 'Flood-prone Areas')
Map.addLayer(building_raster, building_vis, 'All Buildings', False)
Map.addLayer(flooded_buildings, flooded_vis, 'Flooded Buildings')
Map.addLayerControl()

# Display the map
Map

🚀 Loading datasets...
🔍 Calculating flood frequency...
🌊 Identifying flood-prone areas...
🏠 Counting buildings...

📊 Final Results:
Total buildings: 19,367
Flooded buildings: 267

🎨 Generating visualization...


Map(center=[6.746, -57.98], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=SearchDataG…